This notebook implements the assembly of the libre-solar OSH project:

Currently includes only the frame of the enclosure


In [ ]:
# The following should reload if_lib if we change it on disk
%load_ext autoreload
%aimport if_lib, if_utils, if_dpp, if_graphics
%autoreload 1
import os
import json
import random

from if_utils import get_filename, show_data, save_traces

from if_lib import generate_random_challenge, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource_spec_id, get_resource, get_process, create_event, make_transfer, reduce_resource

from if_dpp import trace_query, check_traces, er_before, get_ddp

from if_graphics import vis_dpp, make_sankey, consol_trace

In [ ]:
USE_CASE = 'libre-solar'

# What endpoint are we talking to?
# debug
ENDPOINT = 'http://65.109.11.42:10000/api'
# ENDPOINT = 'http://zenflows-debug.interfacer.dyne.org'
# staging
# ENDPOINT = 'http://65.109.11.42:8000/api'
# ENDPOINT = 'https://zenflows-staging.interfacer.dyne.org/api'
# testing
# ENDPOINT = 'http://65.109.11.42:9000/api'
# ENDPOINT = 'https://zenflows-test.interfacer.dyne.org/api'

USERS_FILE = get_filename('cred_users.json', ENDPOINT, USE_CASE)
LOCS_FILE = get_filename('loc_users.json', ENDPOINT, USE_CASE)
UNITS_FILE = get_filename('units_data.json', ENDPOINT, USE_CASE)
SPECS_FILE = get_filename('res_spec_data.json', ENDPOINT, USE_CASE)

In [ ]:
# This should print something like
# Generated challenge: MMQ1JSrCA7L4QNftLyaaSRunT4Z9+Rr2QkE+a+DWLEljtg6EroLbCj5VjLH+xba9Rv1D+3ncQHw5s/lH41IFJw==
generate_random_challenge()

In [ ]:
# Read or define user data that is going to be used in the GraphQL calls

# create data structure to hold processes
process_data = {}

# create data structures to hold resources and events (possibly to compare results from track and trace)
res_data = {}
event_seq = []


if os.path.isfile(USERS_FILE):
    with open(USERS_FILE,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['one'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "User One",
      "username": "user1_username",
      "email": "user1@example.org",
      "note": "me.user1.org"
    }
    users_data['two'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "User Two",
        "username": "user2",
        "email": "user2@example.org",
        "note" : "me.user2.org"
    }

    with open(USERS_FILE,'w') as f:
        json.dump(users_data, f)


if os.path.isfile(LOCS_FILE):
    with open(LOCS_FILE,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['one'] = {
        "name": "OLVG",
        "lat": 52.35871773455108,
        "long": 4.916762398221842,
        "addr": "Oosterpark 9, 1091 AC Amsterdam",
        "note": "location.user1.org"
    }
    locs_data['two'] = {
        "name": "CleanLease",
        "lat" : 51.47240440868687,
        "long" : 5.412460440524406,
        "addr" : "De schakel 30, 5651 Eindhoven",
        "note": "location.user2.org"
    }
    with open(LOCS_FILE,'w') as f:
        json.dump(locs_data, f)


if os.path.isfile(UNITS_FILE):
    with open(UNITS_FILE,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)



if os.path.isfile(SPECS_FILE):
    with open(SPECS_FILE,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}
  

In [ ]:
# Read HMAC or get it from the server
read_HMAC(USERS_FILE, users_data, 'one', endpoint=ENDPOINT)
read_HMAC(USERS_FILE, users_data, 'two', endpoint=ENDPOINT)

In [ ]:
# Read the keypair

read_keypair(USERS_FILE, users_data, 'one')
read_keypair(USERS_FILE, users_data, 'two')

In [ ]:
# read or get id of the person
get_id_person(USERS_FILE, users_data, 'one', endpoint=ENDPOINT)
get_id_person(USERS_FILE, users_data, 'two', endpoint=ENDPOINT)

In [ ]:
# Read of get the location id
get_location_id(LOCS_FILE, users_data['one'], locs_data, 'one', endpoint=ENDPOINT)
get_location_id(LOCS_FILE, users_data['two'], locs_data, 'two', endpoint=ENDPOINT)

In [ ]:
# Get the ids of all units
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'piece', 'u_piece', 'om2:one', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'mass', 'kg', 'om2:kilogram', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'volume', 'lt', 'om2:litre', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['one'], units_data, 'time', 'h', 'om2:hour', endpoint=ENDPOINT)


In [ ]:
# RESOURCE SPECIFICATIONS
# Read all the resource specifications
name = 'Allen_Key'
note = 'Specification for tool for assembly'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6nutplate'
note = 'Specification for screw'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6x12'
note = 'Specification for screw'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6x25cyl'
note = 'Specification for screw'
classification = 'A resource specification URI'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'RAConnector'
note = 'Specification for connector'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot16'
note = 'Specification for a frame slot'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot32'
note = 'Specification for a frame slot'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot32con'
note = 'Specification for a frame slot'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'assembly_construction'
note = 'Specification for Frame assembly work'
classification = 'A resource specification URI'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'upper_frame'
note = 'Specification for upperframe'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


# **** OLD Read all the resource specifications
name = 'soap'
note = 'Specification for soap to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q34396'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'water'
note = 'Specification for water to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q283'
default_unit_id = units_data['volume']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'cotton'
note = 'Specification for cotton to be used to sew the gowns'
classification = 'https://www.wikidata.org/wiki/Q11457'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'gown'
note = 'Specification for gowns'
classification = 'https://www.wikidata.org/wiki/Q89990310'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'surgical_operation'
note = 'Specification for surgical operations'
classification = 'https://www.wikidata.org/wiki/Q600236'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# RESOURCES
# We create the resources that will not be saved to file as it is assumed they are recreated at each run

res_name = 'Allen_Key'
amount = 1

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6nutplate'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6x12'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6x25cyl'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'RAConnector'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot16'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot32'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot32con'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
# TODO: remove isogown resources
# old isogowns entries left for now ....
res_name = 'soap'
amount = 100

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'water'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'cotton'
amount = 20

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
# PROCESSES
# Create the process that wraps sewing of the gown (its creation)
# print(process_data)
process_name = 'UpperFrame'
user_data = users_data['one']
note = f"Assemble UpperFrame performed by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# TODO: remove isogown processes
# old isogowns.ipynb processes left for now ...
# Create the process that wraps sewing of the gown (its creation)
# print(process_data)
process_name = 'Sew_gown'
user_data = users_data['one']
note = f"Sew gown process performed by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# Create the process that wraps using the gown in the hospital and make it dirty
process_name = 'Use_gown'
user_data = users_data['one']
note = f"Use gown process performed by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# Create the process that includes cleaning the gown
process_name = 'Clean_gown'
user_data = users_data['two']
note = f"Clean gown process performed by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# EVENTS
# Define input consume Tslot32con event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot32con event for Upper Frame Assesmbly'
amount = 1
cur_pros = process_data['UpperFrame']
cur_res = res_data['Tslot32con_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Define input consume Tslot16 event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot16 event for Upper Frame Assesmbly'
amount = 1
cur_pros = process_data['UpperFrame']
cur_res = res_data['Tslot16_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Define input consume M6x25cyl event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume M6x25cyl event for Upper Frame Assesmbly'
amount = 1
cur_pros = process_data['UpperFrame']
cur_res = res_data['M6x25cyl_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Define input event work with the Allen Key to assemble UpperFrame
action = 'work'
event_note='Input work for Upper Frame Assesmbly'
amount = 0.05
cur_pros = process_data['UpperFrame']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Define output event produce for UpperFrame
action = 'produce'
event_note='produce UpperFrame'
amount = 1
cur_pros = process_data['UpperFrame']

res_data['upper_frame_res'] = {
    "res_ref_id": f'upper_frame-{random.randint(0, 10000)}',
    "name": 'upper_frame',
    "spec_id": res_spec_data['upper_frame']['id']
}
cur_res = res_data['upper_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# TODO remove isogown actions
# Define event consume for the gown creation
action = 'consume'
event_note='consume cotton for sewing'
amount = 10
cur_pros = process_data['Sew_gown']
cur_res = res_data['cotton_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Define event produce for the gown creation
action = 'produce'
event_note='produce gown'
amount = 1
cur_pros = process_data['Sew_gown']

res_data['gown_res'] = {
    "res_ref_id": f'gown-{random.randint(0, 10000)}',
    "name": 'gown',
    "spec_id": res_spec_data['gown']['id']
}
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the gown from the owner/leaser to the hospital
note='Transfer gowns to hospital'
action = 'transfer-custody'
amount = 1
cur_res = res_data['gown_res']

event_id, ts = make_transfer(users_data['two'], action, note, users_data['one'], amount, cur_res,  locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Work with the gown to perform surgery
action = 'work'
event_note='work perform surgery'
amount = 80
cur_pros = process_data['Use_gown']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['surgical_operation']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['surgical_operation']['id']

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
    


In [ ]:
# Use the gown to perform surgery and as a consequence make it dirty
action = 'accept'
event_note='accept use for surgery'
amount = 1
cur_pros = process_data['Use_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})
    


In [ ]:
# Modify the gown and make it dirty as a consequence of being used
action = 'modify'
event_note='modify dirty after use'
amount = 1
cur_pros = process_data['Use_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the gown to the leaser for cleaning
note='Transfer gowns to cleaner'
action = 'transfer-custody'
amount = 1
cur_res = res_data['gown_res']

event_id, ts = make_transfer(users_data['one'], action, note, users_data['two'], amount, cur_res,  locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# accept the gown for washing
action = 'accept'
event_note='accept gowns to be cleaned'
amount = 1
cur_pros = process_data['Clean_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume water for washing
action = 'consume'
event_note='consume water for the washing'
amount = 25
cur_pros = process_data['Clean_gown']
cur_res = res_data['water_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# consume soap for washing
action = 'consume'
event_note='consume soap for the washing'
amount = 50
cur_pros = process_data['Clean_gown']
cur_res = res_data['soap_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# modify the gown that is now clean
action = 'modify'
event_note='modify clean after washing'
amount = 1
cur_pros = process_data['Clean_gown']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
show_data(users_data, locs_data, res_data, units_data, res_spec_data, process_data, event_seq)

In [ ]:
# example of using show functions
# show_resource(users_data['one'], '061Z9FX8M4BJ3M6JMB7MW7VSER', endpoint=ENDPOINT)
# show_proposal(users_data['one'], '061Z9FX9H8PT0JF16KP25VS8R0', endpoint=ENDPOINT)


In [ ]:
trace_me = res_data['gown_res']['id']
# trace_me = '062HCB36KR2QRY5HPP2RETC9M4'
print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = {}
er_before(trace_me, users_data['one'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

print(json_object)
print(visited)

In [ ]:
be_dpp = get_ddp(trace_me, user_data, endpoint=ENDPOINT)
print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['one'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(USE_CASE, tot_dpp, trace, be_dpp, event_seq)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)
# make_sankey([0,0,1,2,2], [2,3,3,3,4], ['0','1','2','3','4'], [2,1,1,1,1], color_nodes, color_links)

Here we start some example visualisations

In [ ]:
label = ['Operating Expeditures', 
         'Public Safety', 
         'Engineering and Utilities', 
         'Community-Related Services', 
         'Corporate Support',
         'Police', 
         'Fire',
         'Utilities', 
         'Engineering Public Works',
         'Parks and Recreation', 
         'Arts, Culture, and Community Services',
         'Library', 
         'Development, Buildings, and Licensing',
         'Planning, Urban Design, and Sustainability', 
         'Other',
         'Corporate Support', 
         'Debt and Capital (Non-Utility)', 
         'Contingencies and Transfers']

color_node = ['#808B96', 
             '#EC7063', '#F7DC6F', '#48C9B0', '#AF7AC5',
             '#EC7063', '#EC7063',
             '#F7DC6F', '#F7DC6F',
             '#48C9B0', '#48C9B0', '#48C9B0', '#48C9B0', '#48C9B0', '#48C9B0',
             '#AF7AC5', '#AF7AC5', '#AF7AC5']

color_link = ['#EBBAB5', '#FEF3C7', '#A6E3D7', '#CBB4D5',
              '#EBBAB5', '#EBBAB5',
              '#FEF3C7', '#FEF3C7',
              '#A6E3D7', '#A6E3D7', '#A6E3D7', '#A6E3D7', '#A6E3D7', '#A6E3D7',
              '#CBB4D5', '#CBB4D5', '#CBB4D5']


source = [0, 0, 0, 0,
          1, 1,
          2, 2,
          3, 3, 3, 3, 3, 3,
          4, 4, 4]

target = [1, 2, 3, 4, 
          5, 6,
          7, 8, 
          9, 10, 11, 12, 13, 14, 
          15, 16, 17]

value = [484900, 468350, 355300, 306850, 
         339150, 145350, 
         371450, 96900, 
         129200, 80750, 48450, 48450, 32300, 16150, 
         113050, 129200, 64600]

In [ ]:
# data to dict, dict to sankey
import plotly.graph_objects as go
link = dict(source = source, target = target, value = value, color=color_link)
node = dict(label = label, pad=15, thickness=5, color=color_node)
data = go.Sankey(link = link, node=node)
# plot
fig = go.Figure(data)
fig.show()